We would use RNN (LSTM : long short time memory) to perform sentiment analysis.
Keras has IMDb movie reviews that we can use.


In [0]:
from keras.datasets import imdb

In [88]:
vocabulary_size = 5000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print("Loaded dataset with {} training samples and {} testing samples".format(len(x_train), len(x_test)))

Loaded dataset with 25000 training samples and 25000 testing samples


In [89]:
print('---review---')
print(x_train[6])
print('---label---')
print(y_train[6])

---review---
[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [90]:
word2id = imdb.get_word_index()
id2word = {}
for word, i in word2id.items():
  id2word[i] = word

#testing id2word
print ([id2word.get(i, " ") for i in x_train[6]])

['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 'very', 'bland']


Review words : 

['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 'very', 'bland']

# Padding reviews
In order to be fed to the RNN, all reviews must have the same length, which is now set for 500 words. Longer reviews are truncated and shorter ones are padded

In [0]:
from keras.preprocessing import sequence 
max_words = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)



# Building the Model

Input : sequence of words represented by their IDs
Output : binary sentiment


In [92]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


## Training

Here we specify the loss function as well as the optimizer, and evaluation metrics (here, it's just "accuracy")

In [93]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Actual training

In [94]:
batch_size = 64
num_epochs = 3

x_valid, y_valid = x_train[:batch_size], y_train[:batch_size]
x_train2, y_train2 = x_train[batch_size:], y_train[batch_size:]

model.fit(x_train2, y_train2, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=num_epochs)



Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 292s 12ms/step - loss: 0.4501 - acc: 0.7804 - val_loss: 0.2794 - val_acc: 0.9219
Epoch 2/3
24936/24936 [==============================] - 297s 12ms/step - loss: 0.3060 - acc: 0.8757 - val_loss: 0.1897 - val_acc: 0.9219
Epoch 3/3
24936/24936 [==============================] - 296s 12ms/step - loss: 0.3095 - acc: 0.8696 - val_loss: 0.2802 - val_acc: 0.8906


In [95]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.85904


In [120]:
import numpy as np
import nltk
from nltk import word_tokenize

ex_str =  "the best product, amazing, mesmerizing, exciting"
test = []

for word in word_tokenize(ex_str):
  if(word in word2id):
    if (word2id[word]<5000):
      test.append(word2id[word])

ptest=sequence.pad_sequences([test],maxlen=max_words)

model.predict(ptest)

array([[0.8420145]], dtype=float32)

Saving the Model: 


In [0]:
from keras.models import load_model

model.save_weights('my_model_weights.h5')

#model.load_weights('my_model_weights.h5')
